# Conduct a analysis of the performance


In [32]:
import pandas
import duckdb
import plotly.express as px
import plotly.graph_objects as go

# 1.Read & load the results from the databases crash test !

In [2]:
file_path = '/Users/macbook/Development/database_crash_test/benchmarker/database_benchmark_results.csv'
results_df = pandas.read_csv(file_path)
results_df.columns

Index(['query', 'original_query', 'database_type', 'execution_time_ms',
       'cpu_usage_percent', 'memory_usage_mb', 'memory_usage_percent',
       'disk_read_mb', 'disk_write_mb', 'network_in_mb', 'network_out_mb',
       'result_rows', 'result_size_mb', 'failed'],
      dtype='object')

In [3]:
# check if results return failed queries
failed = results_df[results_df['failed'] == True]
failed

#print(failed['original_query'].values)

,query,original_query,database_type,execution_time_ms,cpu_usage_percent,memory_usage_mb,memory_usage_percent,disk_read_mb,disk_write_mb,network_in_mb,network_out_mb,result_rows,result_size_mb,failed
21,"/* Power output changes */ SELECT d1.time, d1....","-- Power output changes\nSELECT\n d1.time,\...",ClickHouseHandler,62.819958,6.516528,838.320312,20.466805,0.0,0.0,0.002612,0.004488,0,0.0,True


In [4]:
# Display initial data info
print("\nDataFrame Info:")
results_df.info()


DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   query                 22 non-null     object 
 1   original_query        22 non-null     object 
 2   database_type         22 non-null     object 
 3   execution_time_ms     22 non-null     float64
 4   cpu_usage_percent     22 non-null     float64
 5   memory_usage_mb       22 non-null     float64
 6   memory_usage_percent  22 non-null     float64
 7   disk_read_mb          22 non-null     float64
 8   disk_write_mb         22 non-null     float64
 9   network_in_mb         22 non-null     float64
 10  network_out_mb        22 non-null     float64
 11  result_rows           22 non-null     int64  
 12  result_size_mb        22 non-null     float64
 13  failed                22 non-null     bool   
dtypes: bool(1), float64(9), int64(1), object(3)
memory usage: 2

In [5]:
print("\nDescriptive Statistics:")
results_df.describe()


Descriptive Statistics:


,execution_time_ms,cpu_usage_percent,memory_usage_mb,memory_usage_percent,disk_read_mb,disk_write_mb,network_in_mb,network_out_mb,result_rows,result_size_mb
count,22.000000,22.000000,22.000000,22.000000,22.0,22.0,22.000000,22.000000,22.000000,22.000000
mean,8853.203199,5.337268,430.088246,14.228960,0.0,0.0,0.001877,0.562511,39031.909091,1.120877
std,40422.731749,7.027358,339.466861,5.215464,0.0,0.0,0.002496,1.736492,70825.570142,1.949297
min,4.889727,0.001995,1.496094,0.146103,0.0,0.0,0.000000,0.000000,0.000000,0.000000
25%,15.210927,0.004409,128.863281,12.584305,0.0,0.0,0.000000,0.000000,1.000000,0.000149
50%,54.085851,2.233626,392.726562,14.307165,0.0,0.0,0.001216,0.002244,75.000000,0.005276
75%,179.286480,6.341376,744.400391,18.173838,0.0,0.0,0.002679,0.006711,8760.000000,0.601627
max,189821.106911,21.210513,860.394531,21.005726,0.0,0.0,0.008762,6.785174,166536.000000,5.082401


# 2. Analyse the results with SQL in duckdb

In [19]:
results_db = duckdb.sql("SELECT * FROM results_df")
results_db

┌───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────┬────────────────────┬────────────────────┬─────────────────┬──────────────────────┬──────────────┬───────────────┬────────────────────┬────────────────────┬─────────────┬────────────────────┬─────────┐
│                                                                                          

In [7]:
results_df.columns 

Index(['query', 'original_query', 'database_type', 'execution_time_ms',
       'cpu_usage_percent', 'memory_usage_mb', 'memory_usage_percent',
       'disk_read_mb', 'disk_write_mb', 'network_in_mb', 'network_out_mb',
       'result_rows', 'result_size_mb', 'failed'],
      dtype='object')

## Overall Performance Analysis (Execution Time)

In [50]:
# get average execution time
avg_execution_time = duckdb.sql(
    """SELECT 
    database_type,
    ROUND(AVG(execution_time_ms), 2) as avg_execution_time_ms,
    ROUND(MEDIAN(execution_time_ms), 2) as median_execution_time_ms,
    ROUND(MIN(execution_time_ms), 2) as min_execution_time_ms, 
    ROUND(MAX(execution_time_ms), 2) as max_execution_time_ms,
    FROM results_df
    GROUP BY database_type
    ORDER BY avg_execution_time_ms
    """
    )
avg_execution_time

┌───────────────────┬───────────────────────┬──────────────────────────┬───────────────────────┬───────────────────────┐
│   database_type   │ avg_execution_time_ms │ median_execution_time_ms │ min_execution_time_ms │ max_execution_time_ms │
│      varchar      │        double         │          double          │        double         │        double         │
├───────────────────┼───────────────────────┼──────────────────────────┼───────────────────────┼───────────────────────┤
│ ClickHouseHandler │                381.78 │                    62.82 │                 37.51 │                1755.7 │
│ DuckDBHandler     │              17324.63 │                    12.83 │                  4.89 │             189821.11 │
└───────────────────┴───────────────────────┴──────────────────────────┴───────────────────────┴───────────────────────┘

In [51]:
# define dark theme
template = 'plotly_dark'

In [ ]:
"""
# grouped median/avg execution time per database
databases = results_df['database_type'].unique().tolist()

avg_median_perf = go.Figure(
    data=[
    go.Bar(name='Average', x=databases, y=avg_execution_time['avg_execution_time_ms']),
    go.Bar(name='Median', x=databases, y=avg_execution_time['median_execution_time_ms'])
])
avg_median_perf.update_layout(
    barmode='group', 
    template=template, 
    title='Average vs Median Execution Time by Database Type')
avg_median_perf.show()
"""

In [ ]:

avg_execution_time_plot = px.bar(
    avg_execution_time, 
    x='database_type', 
    y='avg_execution_time_ms',
    title='Average Execution Time by Database Type',
    template=template,
    #barmode='group',
    width=800
)
avg_execution_time_plot.show()

## Query execution time analysis